# Disparate Impact Remover Example
Examples and explanations for function parameters https://github.com/mbilalzafar/fair-classification/tree/master/disparate_impact.

In [1]:
import sys
import numpy as np
sys.path.append("../")

from aif360.datasets import GermanDataset
from aif360.metrics import ClassificationMetric
from fairensics.data.synthetic_dataset import SyntheticDataset
from fairensics.data.decision_boundary import DecisionBoundary 
from fairensics.methods import FairDisparateImpact, AccurateDisparateImpact
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
# helper function to print evaluations
def print_evaluation(dataset, clf, unprivileged_groups, privileged_groups):
    predictions = clf.predict(dataset)

    metric = ClassificationMetric(dataset,
                                  predictions,
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

    print("p-rule: \t", metric.disparate_impact())
    print('accuracy: \t', metric.accuracy())
    

# 0.1 Load data

In [3]:
gcd = GermanDataset()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

gcd_train, gcd_test = gcd.split([0.7], shuffle=True)

# 0.2  Metric

- _disparate impact / p-rule_

The acceptance rate must be equal for all groups, independent of the true outcome:

$$P_1(\hat{y}=1)=P_2(\hat{y}=1)$$



## 0.2.1 Definition 
- $z$ the protected features
- $X$ unprotected features
- $\hat{y}$ the predicted label
- $\hat{y}=1$ the positive label  (e.g. getting a loan)
- $P_i(\hat{y}=1)$ the probability for members of group _i_ to be assigned the positive label.

The p-rule is a relaxation of disparate impact:

- _p-rule_ definition: $$\frac{P_i(\hat{y}=1)}{P_j(\hat{y}=1)} \geq p$$

## 0.2.2 Method
To satisfy the p-rule, the covariance between decision boundary and protected attributes is minimized:

$$cov(d(x), z) = \mathbb{E}[(z-\bar{z})d(x)] \cdot \mathbb{E}[((z-\bar{z})\hat{d}(x))) ] \approx \frac{1}{N} \sum _i ^N (z_i - \bar{z})d(x_i)$$


Either by optimizing for accuracy under p-rule constraints or by optimizing for fairness under accuracy constraints.


# 0.3 Parameters (for both constraints)

**- loss function [string]**
- 'logreg' (default)
- 'logreg_l2 
- 'svm_linear'

**- sensitive_attributes [list]**
- list of sensitive attributes to apply constraints to, if not given constraints are aplied to all attributes




# 1. Optimizing  for Accuracy Subject to Fairness Constraints
 _c_: the covariance threshold that can be passed to the function

***minimize***
$$L(\theta)$$
***subject to***
$$\frac{1}{N} \sum _i ^N (z_i - \bar{z})d_\theta(x_i) \leq c$$
$$\frac{1}{N} \sum _i ^N (z_i - \bar{z})d_\theta(x_i) \geq -c$$

# 1.1 Parameters

**- sensitive_attrs_to_cov_thresh** [dictionary]
- the _c_ in above equation, covariance threshold for each sensitive attribute _{'attribute1_name':tresh_1, ...}_    



In [4]:
dspim = AccurateDisparateImpact(warn=False)
print("DisparateImpact with fairness constraint:")
dspim.fit(gcd_train)
print_evaluation(gcd_test, dspim, unprivileged_groups, privileged_groups)

DisparateImpact with fairness constraint:
p-rule: 	 1.0657904187315952
accuracy: 	 0.7166666666666667


# 2. Optimizing Fairness Subject to Accuracy
$\gamma$ must be passed when optimizing for fairness and $L(\theta ^*)$ is the loss of the unconstrained classifier computed automatically

minimize
$$\frac{1}{N} \sum _i ^N (z_i - \bar{z})d_\theta(x_i)$$
subject to
$$L(\theta) \leq (1-\gamma)L(\theta ^*)$$

## 2.1 Parameters

**- gamma [number]**
- tradeoff between accuracy and fairness when optimizing for fairness

**- sep_constraint [bool]**
- fine grained accuracy constraint

In [5]:
dspim = FairDisparateImpact(warn=False)
print("DisparateImpact with accuracy constraint:")
dspim.fit(gcd_train)
print_evaluation(gcd_test, dspim, unprivileged_groups, privileged_groups)

DisparateImpact with accuracy constraint:
p-rule: 	 1.0173316616269636
accuracy: 	 0.7266666666666667


../fairensics/methods/disparate_impact.py:364: UserWarning: Only the first protected attribute is considered with this constraint.
  warnings.warn("Only the first protected attribute is considered with this constraint.")
